In [1]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

%load_ext autoreload
%autoreload 2

In [ ]:
import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.DL_architecture as ARCH

# Assign seed
seed = 18

# Paths
data_dir = ROOT / "data" / "tif_files"
h5_dir   = ROOT / "data" / "h5_files"
h5_dir.mkdir(parents=True, exist_ok=True)
h5_file  = h5_dir / "data.h5"

# Build H5 files 
if not h5_file.exists():
    DL.create_h5(str(data_dir), str(h5_dir), upper_threshold=3)
else:
    print(f"Using existing H5: {h5_file}")

Using existing H5: c:\Users\mathi\Documents\Paper2\ML_SD\data\h5_files\data.h5


In [7]:
# --- Config ---
HOLDOUT_AOI  = "ID_BS"   # change if you want a different test AOI
VAL_FRACTION = 0.30
SEED         = seed

# --- H5 path ---
h5_file = Path(h5_dir) / "data.h5"
assert h5_file.exists(), f"Missing H5 file: {h5_file}"

(X_train, y_train), (X_val, y_val), (X_hold, y_hold) = DS.dl_unet_split(
    h5_path=str(h5_file),
    holdout_aoi="ID_BS",
    val_fraction=0.30,
    seed=seed,
    patch_size=128,     # smaller tiles capture clean areas
    stride=64,          # overlap -> more tiles
    min_valid_frac=0.80 # relax coverage
)


print(
    f"Shapes:\n"
    f"  X_train: {X_train.shape}  y_train: {y_train.shape}\n"
    f"  X_val:   {X_val.shape}    y_val:   {y_val.shape}\n"
    f"  X_hold:  {X_hold.shape}   y_hold:  {y_hold.shape}"
)

# --- Per-channel z-score using TRAIN stats only ---
def zscore_from_train(x_train, *xs, eps=1e-6):
    mu  = x_train.mean(axis=(0, 1, 2), keepdims=True)
    std = x_train.std(axis=(0, 1, 2), keepdims=True) + eps
    normed = tuple((x - mu) / std for x in (x_train,) + xs)
    return (mu, std), normed

(mu, std), (X_train_n, X_val_n, X_hold_n) = zscore_from_train(X_train, X_val, X_hold)

# --- Build per-pixel masks from NaNs in SD and fill NaNs with 0 ---
import numpy as np

def fill_nan_and_mask(y):
    # y: (N, H, W, 1)
    mask = (~np.isnan(y))[..., 0].astype("float32")        # (N, H, W) -> 1 where SD is valid
    y_filled = np.where(np.isnan(y), 0.0, y).astype("float32")
    return y_filled, mask

y_train_f, w_train = fill_nan_and_mask(y_train)
y_val_f,   w_val   = fill_nan_and_mask(y_val)
y_hold_f,  w_hold  = fill_nan_and_mask(y_hold)  # for masked metrics later

X_train_n = X_train_n.astype("float32")
X_val_n   = X_val_n.astype("float32")
X_hold_n  = X_hold_n.astype("float32")



Shapes:
  X_train: (212, 128, 128, 7)  y_train: (212, 128, 128, 1)
  X_val:   (43, 128, 128, 7)    y_val:   (43, 128, 128, 1)
  X_hold:  (362, 128, 128, 7)   y_hold:  (362, 128, 128, 1)


In [8]:
print("NaNs in y_filled (train/val/hold):",
      np.isnan(y_train_f).any(), np.isnan(y_val_f).any(), np.isnan(y_hold_f).any())
print("Mask coverage (train/val/hold):",
      w_train.mean(), w_val.mean(), w_hold.mean())


NaNs in y_filled (train/val/hold): False False False
Mask coverage (train/val/hold): 0.9827944 0.97112894 0.99160194


In [ ]:
import tensorflow as tf

# --- Build ---
model = ARCH.unet_regression(
    input_shape=X_train_n.shape[1:],   # (H, W, C)
    base_filters=32
)

# --- Compile ---
LR = 1e-3
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)

model.summary(line_length=120, expand_nested=True)

# --- Train with pixel-wise masks (broadcasts (N,H,W) -> targets (N,H,W,1)) ---
import os, json
os.makedirs("weights", exist_ok=True)

CKPT = "weights/unet_best.weights.h5"
NORM = "weights/unet_norm.json"

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        CKPT, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]

# train (unchanged except callbacks list)
history = model.fit(
    X_train_n, y_train_f,
    sample_weight=w_train,
    validation_data=(X_val_n, y_val_f, w_val),
    epochs=100,
    batch_size=4,
    callbacks=callbacks,
    verbose=1,
)

history = model.fit(
    X_train_n, y_train_f,
    sample_weight=w_train,                         # << mask invalid pixels
    validation_data=(X_val_n, y_val_f, w_val),    # << masked validation
    epochs=100,
    batch_size=4,
    callbacks=callbacks,
    verbose=1,
)


In [10]:
from numpy import sqrt

y_pred_hold = model.predict(X_hold_n)

def masked_rmse_mae(y_true, y_pred, w, eps=1e-6):
    # y_*: (N,H,W,1), w: (N,H,W)
    diff = (y_true[...,0] - y_pred[...,0])
    mae = (np.abs(diff) * w).sum() / (w.sum() + eps)
    rmse = sqrt(((diff**2) * w).sum() / (w.sum() + eps))
    return rmse, mae

rmse, mae = masked_rmse_mae(y_hold_f, y_pred_hold, w_hold)
print(f"Hold-out (masked) — RMSE: {rmse:.4f}, MAE: {mae:.4f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 730ms/step
Hold-out (masked) — RMSE: 0.4799, MAE: 0.3601
